# [STARTER] Exercise - Building an AI Agent with Tools

In this exercise, you'll build an AI agent that can use tools to enhance its capabilities. You'll learn how to create an agent that can understand when to use tools, process their results, and maintain a coherent conversation.

## Challenge

Imagine you're building a smart coding assistant that needs to:
- Answer programming questions
- Execute code snippets
- Look up documentation
- Perform calculations
- Search through codebases

Instead of hard-coding when to use each capability, your agent should intelligently decide when and how to use its available tools.

## Setup
First, let's import the necessary libraries:

In [2]:
from typing import List, Dict, Any
from dotenv import load_dotenv
from copy import deepcopy
import json
import os

from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM

## Understanding the Components

Before we build our agent, let's understand the key components we'll be working with:

- `LLM`: The language model wrapper that handles tool execution
- `SystemMessage`: Defines the agent's role and behavior
- `UserMessage`: Represents user inputs
- `ToolMessage`: Contains tool execution results
- `tool`: Decorator for creating tools

## Building the Agent Class

Your task is to create an Agent class that can:
1. Initialize with a specific role and set of tools
2. Process user messages
3. Decide when to use tools
4. Handle tool responses

In [3]:
class Agent:
    """An AI Agent that can use tools to help answer questions"""
    
    def __init__(
        self,
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[Any] = None,
    ):
        """Initialize the agent with its configuration and tools"""
        # TODO 1: Initialize the agent
        # Hint: 
        # - Load environment variables with load_dotenv()
        load_dotenv()
        self.api_key = os.getenv("OPENAI_API_KEY")
        # - Store agent settings (role, instructions, etc.)
        self.role = role,
        self.instructions = instructions,
        self.model = model,
        self.temperature = temperature,
        self.tools = tools
        
        # - Create an LLM instance with the provided tools
        self.llm = LLM(model, temperature, tools, api_key=self.api_key)

    def invoke(self, user_message: str) -> str:
        """Process a user message and return a response"""
        # TODO 2: Set up the conversation
        # Hint:
        # - Create messages list with SystemMessage (role + instructions)
        messages = [
            SystemMessage(content=f"You're a {self.role}. {self.instructions}"),
            UserMessage(content=user_message)
            ]
        ai_message = self.llm.invoke(messages)
        messages.append(ai_message)
        # - Add UserMessage with user_message
        # - Get initial AI response using self.llm.invoke()

        # TODO 3: Handle tool calls if needed
        # Hint:
        # - Check if ai_message.tool_calls exists
        # - For each tool call:
        #   * Get function name and arguments
        #   * Execute tool and get result
        #   * Add result as ToolMessage
        # - Get final AI response
        if ai_message.tool_calls:
            for tool_call in ai_message.tool_calls:
                tool_call_id = tool_call.id
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                tool = next((t for t in self.tools if t.name == function_name), None)
                if tool:
                    result = tool(**function_args)
                    messages.append(
                        ToolMessage(
                            content=json.dumps(result), 
                            tool_call_id=tool_call_id, 
                            name=function_name, 
                        )
                    )
            ai_message = self.llm.invoke(messages)
            messages.append(ai_message)
            
        
        for message in messages:
            print(message)
        return ai_message.content

## Testing Your Agent

Once you've implemented the Agent class, test it with different scenarios:

1. Basic conversation without tools

In [4]:
agent = Agent(role="Coding Assistant")
response = agent.invoke("What is Python? Be concise")
print(response)

content="You're a ('Coding Assistant',). ('Help users with any question',)" role='system'
content='What is Python? Be concise' role='user'
content='Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data analysis, artificial intelligence, scientific computing, and automation, among other applications.' role='assistant' tool_calls=None
Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data analysis, artificial intelligence, scientific computing, and automation, among other applications.


2. Create a calculator tool

In [5]:
@tool
def calculate(expression: str) -> float:
    """Evaluate a mathematical expression"""
    return eval(expression)

In [6]:
# Create an agent with the calculator tool
math_agent = Agent(
    role="Math Assistant",
    tools=[calculate]
)

In [7]:
response = math_agent.invoke("What is 23 * 45?")
print(response)

content="You're a ('Math Assistant',). ('Help users with any question',)" role='system'
content='What is 23 * 45?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_nGbB945pEhhL7MJDzOtU6gy0', function=Function(arguments='{"expression":"23 * 45"}', name='calculate'), type='function')]
content='1035' role='tool' tool_call_id='call_nGbB945pEhhL7MJDzOtU6gy0' name='calculate'
content='The result of \\( 23 \\times 45 \\) is 1035.' role='assistant' tool_calls=None
The result of \( 23 \times 45 \) is 1035.


In [8]:
# Test multiple tool usage
response = math_agent.invoke("If I multiply 3 by 5, what do I get? Then later add 7")
print(response)

content="You're a ('Math Assistant',). ('Help users with any question',)" role='system'
content='If I multiply 3 by 5, what do I get? Then later add 7' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_DxGNYaUNZpWGhoNiAQUmW9r9', function=Function(arguments='{"expression": "3 * 5"}', name='calculate'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_IxzFYowFapYv8C9NgffeJuNZ', function=Function(arguments='{"expression": "3 * 5 + 7"}', name='calculate'), type='function')]
content='15' role='tool' tool_call_id='call_DxGNYaUNZpWGhoNiAQUmW9r9' name='calculate'
content='22' role='tool' tool_call_id='call_IxzFYowFapYv8C9NgffeJuNZ' name='calculate'
content='If you multiply 3 by 5, you get 15. If you then add 7 to that result, you get 22.' role='assistant' tool_calls=None
If you multiply 3 by 5, you get 15. If you then add 7 to that result, you get 22.


3. Create a data analyst

In [10]:
@tool
def get_games(num_games:int=1, top:bool=True) -> str:
    """
    Returns the top or bottom N games with highest or lowest scores.    
    args:
        num_games (int): Number of games to return (default is 1)
        top (bool): If True, return top games, otherwise return bottom (default is True)
    """
    data = [
        {"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98},
        {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97},
        {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97},
        {"Game": "Super Smash Bros. Brawl", "Platform": "Wii", "Score": 93},
        {"Game": "Mario Kart 8 Deluxe", "Platform": "Switch", "Score": 92},
        {"Game": "Fire Emblem: Awakening", "Platform": "3DS", "Score": 92},
        {"Game": "Donkey Kong Country Returns", "Platform": "Wii", "Score": 87},
        {"Game": "Luigi's Mansion 3", "Platform": "Switch", "Score": 86},
        {"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85},
        {"Game": "Animal Crossing: New Leaf", "Platform": "3DS", "Score": 88}
    ]
    # Sort the games list by Score
    # If top is True, descending order
    sorted_games = sorted(data, key=lambda x: x['Score'], reverse=top)
    
    # Return the N games
    return sorted_games[:num_games]

In [11]:
# Create an agent with the multiple tools
data_analyst_agent = Agent(
    role="Game Stats Assistant",
    instructions="You can bring insights about a game dataset based on users questions",
    tools=[get_games]
)

In [12]:
response = data_analyst_agent.invoke("What's the best game in the dataset?")
print(response)

content="You're a ('Game Stats Assistant',). ('You can bring insights about a game dataset based on users questions',)" role='system'
content="What's the best game in the dataset?" role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_yT332wSGT3yffmebfeylWw29', function=Function(arguments='{"num_games":1,"top":true}', name='get_games'), type='function')]
content='[{"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98}]' role='tool' tool_call_id='call_yT332wSGT3yffmebfeylWw29' name='get_games'
content='The best game in the dataset is **The Legend of Zelda: Breath of the Wild** for the Switch, with a score of **98**.' role='assistant' tool_calls=None
The best game in the dataset is **The Legend of Zelda: Breath of the Wild** for the Switch, with a score of **98**.


In [13]:
response = data_analyst_agent.invoke("What's the worst game in the dataset?")
print(response)

content="You're a ('Game Stats Assistant',). ('You can bring insights about a game dataset based on users questions',)" role='system'
content="What's the worst game in the dataset?" role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_PbMXkYzFTbQpMoy4huQvqU1m', function=Function(arguments='{"num_games":1,"top":false}', name='get_games'), type='function')]
content='[{"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85}]' role='tool' tool_call_id='call_PbMXkYzFTbQpMoy4huQvqU1m' name='get_games'
content='The worst game in the dataset is "Pikmin 3" for the Wii U, with a score of 85.' role='assistant' tool_calls=None
The worst game in the dataset is "Pikmin 3" for the Wii U, with a score of 85.


In [15]:
# Create an agent with the multiple tools
game_stats_agent = Agent(
    role="Game Stats Assistant",
    instructions="You can bring insights about a game dataset based on users questions",
    tools=[get_games, calculate]
)

In [16]:
response = game_stats_agent.invoke("What is the average score?")
print(response)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)

In [ ]:
response = game_stats_agent.invoke("What is the distribution of platforms?")
print(response)